In [100]:
import pandas as pd
pd.set_option('display.max_columns', None)

drev_lots = pd.read_csv("../../web/exports_igpgascogne/drev_lots.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Siret Opérateur': 'str', 'Code postal Opérateur': 'str', 'Millésime':'str'}, low_memory=False)
etablissements = pd.read_csv("../../web/exports_igpgascogne/etablissements.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Login': 'str', 'Identifiant etablissement': 'str'}, index_col=False, low_memory=False)
societe = pd.read_csv("../../web/exports_igpgascogne/societe.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Téléphone' :'str', 'Téléphone portable': 'str'}, index_col=False, low_memory=False)
lots = pd.read_csv("../../web/exports_igpgascogne/lots.csv", encoding="iso8859_15", delimiter=";", decimal=",", index_col=False, low_memory=False)


In [101]:
#print(drev_lots['Date lot'][1] > '2021')
drev_lots = drev_lots.rename(columns = {'Date lot': 'Date_lot'})

#print(drev_lots)


In [102]:
millesime = "2019"
datemax = "2021"
drev_lots = drev_lots.query("Millésime == @millesime");
drev_lots = drev_lots.query("Date_lot < @datemax")
drev_lots['Lieu'] = drev_lots['Lieu'].fillna('')
drev_lots = drev_lots.groupby(['Appellation','Couleur','Lieu'])[["Volume"]].sum()
drev_lots = drev_lots.reset_index()

In [103]:
print(drev_lots)

   Appellation     Couleur Lieu     Volume
0          CDG       blanc       564897.15
1          CDG  blanc_doux        87009.39
2          CDG        rose        70679.10
3          CDG        rose  CON    6505.00
4          CDG       rouge        63751.03
5          CDG       rouge  CON    6980.00
6          COT       blanc         4671.00
7          COT        rose        48088.00
8          COT       rouge         5695.50
9          GER       blanc        11776.00
10         GER        rose           49.00
11         GER       rouge          350.00
